In [1]:
import ipywidgets as wd
wd.IntSlider()


IntSlider(value=0)

In [2]:
# --- 1. Librerie ---------------------------------------------
from pathlib import Path, PurePosixPath
import os
from PIL import Image
from io import BytesIO
import ipywidgets as widgets
from IPython.display import HTML, display
import uuid, ipywidgets as widgets
import ipywidgets as w, uuid, pathlib
from IPython.display import HTML, display


frames_dir = Path("./anim_frames_7001358")
frame_paths = sorted(frames_dir.glob("*.png"))    

video_path = "ciclone7001358.mp4" 


abs_path   = Path(video_path).resolve() # /media/isacDisk1/VideoMAEv2/ciclone7001358.mp4
server_root = Path(os.environ.get("PWD", ".")).resolve()   # dove hai lanciato Jupyter
rel_path   = abs_path.relative_to(server_root)             # media/isacDisk1/VideoMAEv2/...
src_url    = f"/files/{PurePosixPath(rel_path).as_posix()}"  # /files/media/...

print("DEBUG — URL al browser:", src_url)    # verifica

DEBUG — URL al browser: /files/VideoMAEv2/ciclone7001358.mp4


In [3]:
# ── 2. widget Python che conterranno i timestamp ───────────────
t_in  = wd.FloatText(description="INIZIO", disabled=True)
t_out = wd.FloatText(description="FINE",   disabled=True)

# ── 3. id HTML univoci ─────────────────────────────────────────
vid_id, b1_id, b2_id = [f"{p}_{uuid.uuid4().hex[:6]}" for p in ("vid","b1","b2")]

# ── 4. HTML + ponte JS basato su window.require ────────────────
html = f"""
<video id="{vid_id}" width="720" controls src="{src_url}"></video><br>
<button id="{b1_id}">Segna&nbsp;INIZIO</button>
<button id="{b2_id}">Segna&nbsp;FINE</button>

<script>
// attende che window.require sia caricato (lo porta JupyterLab stesso)
(function waitRequire(cb){{
  if (window.require) return cb();
  setTimeout(()=>waitRequire(cb), 50);
}})(function(){{
  window.require(['@jupyter-widgets/base'], function(widgets){{
      const mgr = [...widgets.ManagerBase._managers][0];
      const vid = document.getElementById('{vid_id}');
      document.getElementById('{b1_id}').onclick = () => {{
          mgr.get_model('{t_in.model_id}').then(m=>{{m.set('value',vid.currentTime);
          console.log("scritto", vid.currentTime, "nel widget");
          m.save_changes();}});
      }};
      document.getElementById('{b2_id}').onclick = () => {{
          mgr.get_model('{t_out.model_id}').then(m=>{{m.set('value',vid.currentTime);m.save_changes();}});
      }};
  }});
}});
</script>
"""

display(HTML(html))
display(t_in, t_out)


FloatText(value=0.0, description='INIZIO', disabled=True)

FloatText(value=0.0, description='FINE', disabled=True)

In [4]:
print(t_in.value, t_out.value)


0.0 0.0


In [3]:


# ── widget Python per inizio / fine (readonly lato utente) ───────────────────
t_in  = wd.FloatText(description="INIZIO", disabled=True)
t_out = wd.FloatText(description="FINE",   disabled=True)

# ── id univoci per video e pulsanti ───────────────────────────────────────────
vid_id, b_in, b_out = [f"{p}_{uuid.uuid4().hex[:6]}" for p in ("vid","bin","bout")]

html = f"""
<video id="{vid_id}" width="720" controls src="{src_url}"></video><br>
<button id="{b_in}">Segna INIZIO</button>
<button id="{b_out}">Segna FINE</button>

<script>
/* 1️⃣  aspetta che la funzione require() sia disponibile nell’output */
(function wait(cb){{
  if (window.require) cb();
  else setTimeout(()=>wait(cb), 50);
}})(function(){{
  /* 2️⃣  carica @jupyter-widgets/base **nella stessa versione del manager**
         (5.0.15, la stessa che hai in labextension list)                 */
  require(['@jupyter-widgets/base'], function(widgets){{
      const mgr = [...widgets.ManagerBase._managers][0];   // ora esiste
      const vid = document.getElementById('{vid_id}');

      const setValue = (modelId, label) => {{
        mgr.get_model(modelId).then(m => {{
            const t = +vid.currentTime.toFixed(3);
            m.set('value', t); m.save_changes();
            console.log(label, t);          // LOG in console
        }});
      }};

      document.getElementById('{b_in}') .onclick = () => setValue('{t_in.model_id}',  '⟲ INIZIO');
      document.getElementById('{b_out}').onclick = () => setValue('{t_out.model_id}', '⟳ FINE');
  }});
}});
</script>
"""

display(HTML(html))
display(t_in, t_out)


FloatText(value=0.0, description='INIZIO', disabled=True)

FloatText(value=0.0, description='FINE', disabled=True)